In [129]:
# Load Library

packages <- c("dplyr", "haven","ggplot2")

install_if_missing <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg)
  }
  library(pkg, character.only = TRUE)
}

lapply(packages, install_if_missing)

#test


[[1]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[2]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[3]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"

# Read Data

## Helper Function

In [60]:
compare_variables <- function(ds1, ds2) {
  vars_ds1 <- colnames(ds1)
  vars_ds2 <- colnames(ds2)
  
  matched_vars <- intersect(vars_ds1, vars_ds2)
  only_in_ds1 <- setdiff(vars_ds1, vars_ds2)
  only_in_ds2 <- setdiff(vars_ds2, vars_ds1)

  list(
    matched = matched_vars,
    only_in_ds1 = only_in_ds1,
    only_in_ds2 = only_in_ds2
  )
}


compare_variable_list <- function(dataset, var_list) {
  dataset_vars <- colnames(dataset)
  
  matched_vars <- intersect(dataset_vars, var_list)
  only_in_dataset <- setdiff(dataset_vars, var_list)
  only_in_list <- setdiff(var_list, dataset_vars)
  
  return(list(
    matched = matched_vars,
    only_in_dataset = only_in_dataset,
    only_in_list = only_in_list
  ))
}


compare_list <- function(var_list1, var_list2) {
  
  matched_vars <- intersect(var_list1, var_list2)
  only_in_list1 <- setdiff(var_list1, var_list2)
  only_in_list2 <- setdiff(var_list2, var_list1)
  
  return(list(
    matched = matched_vars,
    only_in_list1 = only_in_list1,
    only_in_list2 = only_in_list2
  ))
}


## Selected Variables

In [130]:
var9902 <- readLines("Data/var9902.txt") # Selected Variable for Diet
length(var9902)
var0304 <- readLines("Data/var0304.txt") # Selected Variable for Diet
length(var0304)
var0520 <- readLines("Data/var0520.txt") # Selected Variable for Diet
length(var0520)

var_nhanes <- readLines("Data/Nhanes_Variables.txt") # Selected Variable for Nhanes
length(var_nhanes)

Warning message in readLines("Data/var9902.txt"):
"incomplete final line found on 'Data/var9902.txt'"


[1] 46

Warning message in readLines("Data/var0304.txt"):
"incomplete final line found on 'Data/var0304.txt'"


[1] 46

Warning message in readLines("Data/var0520.txt"):
"incomplete final line found on 'Data/var0520.txt'"


[1] 46

[1] 10

In [123]:
# compare_list(var9902, var0304)
# compare_list(var0304, var0520)

## Read Nhanes Data

In [124]:
nahnes <- read.csv("Data/nhanes_data.csv") %>% 
    select(all_of(var_nhanes))
head(nahnes)


,svy_id,demo_age_cat,demo_race,demo_race_black,demo_age_years,demo_pregnant,demo_gender,bp_sys_mean,bp_dia_mean,cc_bmi
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,12,18 to 44,Non-Hispanic White,No,37,No,Men,176.6667,102.00000,30 to <35
2,21,18 to 44,Hispanic,No,18,No,Men,121.3333,80.00000,35+
3,27,18 to 44,Hispanic,No,18,No,Men,118.0000,78.00000,25 to <30
4,28,18 to 44,Non-Hispanic White,No,18,No,Men,94.0000,42.66667,<25
5,56,18 to 44,Non-Hispanic White,No,21,No,Men,121.0000,65.00000,<25
6,57,18 to 44,Non-Hispanic White,No,39,No,Men,119.3333,85.33333,25 to <30


## Read DT1TOT Data

In [122]:
# DR1TOT
tot <- data.frame() 

## DR1TOT_A Have different wate variable name, rename it
tot_a <- read_xpt("Data/DR1TOT_A.xpt")
names(tot_a)[names(tot_a) == "DRXTWATE"] <- "DRXTMOIS"
tot_a <- tot_a  %>% 
    select(var9902)

tot <- rbind(tot, tot_a)


## DR1TOT_B
tot_b <- read_xpt("Data/DR1TOT_B.xpt")
tot_b <- tot_b  %>% 
    select(var9902)

tot <- rbind(tot, tot_b)


## DR1TOT_C
tot_c <- read_xpt("Data/DR1TOT_C.xpt")
tot_c <- tot_c  %>% 
    select(all_of(var0304))
names(tot_c) <- var9902

tot <- rbind(tot, tot_c)


## DR1TOT_D to DR1TOT_J
files <- paste0("Data/DR1TOT_", LETTERS[4:10], ".xpt")  

for (file in files) {
    temp <- read_xpt(file) %>%
        select(all_of(var0520))
    names(temp) <- var9902  
    tot <- rbind(tot, temp) 
}

# View Data
head(tot)
tail(tot)

SEQN,DRXTPROT,DRXTCARB,DRXTTFAT,DRXTSFAT,DRXTMFAT,DRXTPFAT,DRXTCHOL,DRXTFIBE,DRXTVB6,⋯,DRXTM201,DRXTM221,DRXTP182,DRXTP183,DRXTP184,DRXTP204,DRXTP205,DRXTP225,DRXTP226,DRD320GW
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,31.96,250.36,27.24,9.40,9.00,6.40,46.55,7.41,1.43,⋯,0.03,0.00,6.07,0.27,0,0.03,0.00,0.00,0.00,243.38
2,123.16,350.37,71.95,24.34,26.48,11.94,313.95,36.99,2.90,⋯,0.09,0.02,10.07,1.14,0,0.26,0.00,0.01,0.02,0.00
3,40.19,233.63,49.94,15.61,19.57,11.84,86.22,11.16,1.37,⋯,0.16,0.00,11.12,0.55,0,0.07,0.01,0.02,0.03,531.00
4,56.16,191.03,56.20,27.54,20.13,4.67,194.78,5.45,0.95,⋯,0.05,0.14,3.96,0.66,0,0.05,0.00,0.00,0.00,1652.00
5,97.13,253.98,114.52,36.00,60.55,11.35,180.57,17.28,2.92,⋯,0.18,0.00,10.03,1.25,0,0.07,0.00,0.01,0.01,1298.00
6,24.63,202.11,25.15,7.78,7.84,7.15,38.45,6.24,0.97,⋯,0.05,0.01,6.41,0.69,0,0.03,0.00,0.00,0.00,1711.00


SEQN,DRXTPROT,DRXTCARB,DRXTTFAT,DRXTSFAT,DRXTMFAT,DRXTPFAT,DRXTCHOL,DRXTFIBE,DRXTVB6,⋯,DRXTM201,DRXTM221,DRXTP182,DRXTP183,DRXTP184,DRXTP204,DRXTP205,DRXTP225,DRXTP226,DRD320GW
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
124817,51.33,90.33,63.18,25.360,22.624,9.509,813,5.8,0.691,⋯,0.106,0.000,8.038,0.832,0.001,0.392,0.003,0.025,0.111,1521.0
124818,58.05,512.14,183.51,62.115,61.034,45.841,220,15.0,0.647,⋯,0.444,0.004,41.362,4.263,0.000,0.088,0.010,0.019,0.006,0.0
124819,67.43,197.26,77.71,32.515,24.228,10.880,175,8.3,1.421,⋯,0.164,0.001,9.937,0.981,0.002,0.070,0.007,0.018,0.003,225.0
124820,39.32,157.81,48.77,15.558,13.070,14.765,84,12.3,2.002,⋯,0.138,0.004,13.410,1.293,0.002,0.042,0.007,0.009,0.001,0.0
124821,138.10,110.59,76.03,15.733,30.469,20.797,484,6.7,1.688,⋯,0.546,0.020,17.314,1.944,0.009,0.309,0.124,0.090,0.350,1267.5
124822,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


## Merge Nhanes and DT1TOT

In [128]:
merge_df <- merge(nahnes, tot, by.x = "svy_id", by.y = "SEQN")

head(merge_df)
tail(merge_df)

,svy_id,demo_age_cat,demo_race,demo_race_black,demo_age_years,demo_pregnant,demo_gender,bp_sys_mean,bp_dia_mean,cc_bmi,⋯,DRXTM201,DRXTM221,DRXTP182,DRXTP183,DRXTP184,DRXTP204,DRXTP205,DRXTP225,DRXTP226,DRD320GW
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2,75+,Non-Hispanic White,No,77,No,Men,100.6667,56.66667,<25,⋯,0.09,0.02,10.07,1.14,0.00,0.26,0.00,0.01,0.02,0
2,5,45 to 64,Non-Hispanic White,No,49,No,Men,122.0000,82.66667,25 to <30,⋯,0.18,0.00,10.03,1.25,0.00,0.07,0.00,0.01,0.01,1298
3,6,18 to 44,Other,No,19,No,Women,114.6667,68.00000,<25,⋯,0.05,0.01,6.41,0.69,0.00,0.03,0.00,0.00,0.00,1711
4,7,45 to 64,Non-Hispanic Black,Yes,59,No,Women,125.3333,80.00000,25 to <30,⋯,0.08,0.01,15.31,2.23,0.06,0.13,0.15,0.03,0.20,236
5,10,18 to 44,Non-Hispanic Black,Yes,43,No,Men,145.3333,96.00000,30 to <35,⋯,0.20,0.01,12.73,2.06,0.00,0.28,0.00,0.00,0.00,1888
6,12,18 to 44,Non-Hispanic White,No,37,No,Men,176.6667,102.00000,30 to <35,⋯,0.31,0.33,35.46,3.46,0.00,0.49,0.01,0.00,0.09,3304


,svy_id,demo_age_cat,demo_race,demo_race_black,demo_age_years,demo_pregnant,demo_gender,bp_sys_mean,bp_dia_mean,cc_bmi,⋯,DRXTM201,DRXTM221,DRXTP182,DRXTP183,DRXTP184,DRXTP204,DRXTP205,DRXTP225,DRXTP226,DRD320GW
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
59794,124814,45 to 64,Non-Hispanic Black,Yes,64,No,Men,140.1667,85.36667,35+,⋯,0.211,0.008,16.079,1.911,0.000,0.085,0.005,0.009,0.005,1014.0
59795,124815,45 to 64,Non-Hispanic Black,Yes,52,No,Men,141.5000,85.36667,25 to <30,⋯,0.403,0.066,13.748,1.040,0.106,0.455,0.008,0.030,0.093,2028.0
59796,124817,65 to 74,Hispanic,No,67,No,Women,113.5000,66.03333,35+,⋯,0.106,0.000,8.038,0.832,0.001,0.392,0.003,0.025,0.111,1521.0
59797,124818,18 to 44,Non-Hispanic Black,Yes,40,No,Men,104.5000,66.03333,35+,⋯,0.444,0.004,41.362,4.263,0.000,0.088,0.010,0.019,0.006,0.0
59798,124821,45 to 64,Non-Hispanic Black,Yes,63,No,Men,125.5000,65.36667,25 to <30,⋯,0.546,0.020,17.314,1.944,0.009,0.309,0.124,0.090,0.350,1267.5
59799,124822,65 to 74,Hispanic,No,74,No,Men,123.5000,55.03333,<25,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
